In [1]:
spark

res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@50bdfae5


In [39]:
val user_episode_df = spark.read.parquet("/Users/mthota/Dropbox/Data/final_datasets/users_program_data/")
val tivo_prog_df = spark.read.option("header","true").csv("/Users/mthota/Dropbox/Data/final_datasets/feature_enginnering/program_features/data_cleaned/csv/GenreImputed/ProgramMetadata.csv")
val tivo_imdb_wiki_df =spark.read.parquet("/Users/mthota/Dropbox/Data/final_datasets/feature_enginnering/program_features/program_features_tivo_imdb_wiki/")

user_episode_df: org.apache.spark.sql.DataFrame = [PROGRAM_ID: string, PARENT_PROGRAM_ID: string ... 23 more fields]
tivo_prog_df: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, EVENT_DATE: string ... 18 more fields]
tivo_imdb_wiki_df: org.apache.spark.sql.DataFrame = [MASTER_TITLE: string, EVENT_DATE: string ... 25 more fields]


In [46]:
user_episode_df.printSchema()
tivo_prog_df.printSchema()
tivo_imdb_wiki_df.printSchema()
user_episode_df.createOrReplaceTempView("user_episode_df")
tivo_prog_df.createOrReplaceTempView("tivo_prog_df")
tivo_imdb_wiki_df.createOrReplaceTempView("tivo_imdb_wiki_df")

root
 |-- PROGRAM_ID: string (nullable = true)
 |-- PARENT_PROGRAM_ID: string (nullable = true)
 |-- PREV_PROGRAM_ID: string (nullable = true)
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- SK_DEVICE_ID: integer (nullable = true)
 |-- _c9: long (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: integer (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable = true)
 |-- airEndTime: string (nul

In [16]:
val user_episode_watch_df  = spark.sql("SELECT PROGRAM_ID,PARENT_PROGRAM_ID,MASTER_TITLE,SK_DEVICE_ID as USER,CASE WHEN _c9 > RUNTIME THEN RUNTIME ELSE _c9 END as EPISODE_WATCH_DURATION,RUNTIME AS EPISODE_DURATION,(CASE WHEN _c9 > RUNTIME THEN RUNTIME ELSE _c9 END)/RUNTIME AS WATCH_RATIO FROM user_episode_df") 

user_episode_watch_df: org.apache.spark.sql.DataFrame = [PROGRAM_ID: string, PARENT_PROGRAM_ID: string ... 5 more fields]


In [17]:
user_episode_watch_df.show(10,false)
user_episode_watch_df.createOrReplaceTempView("user_episode_watch_df")

+----------+-----------------+-------------------------------+-------+----------------------+----------------+------------------+
|PROGRAM_ID|PARENT_PROGRAM_ID|MASTER_TITLE                   |USER   |EPISODE_WATCH_DURATION|EPISODE_DURATION|WATCH_RATIO       |
+----------+-----------------+-------------------------------+-------+----------------------+----------------+------------------+
|14745095  |11533645         |The Good Doctor                |5118788|2505                  |3600            |0.6958333333333333|
|14653884  |null             |One Winter Weekend             |5161679|7200                  |7200            |1.0               |
|14962564  |6718065          |The Big Bang Theory            |5946021|1800                  |1800            |1.0               |
|14802512  |7116387          |Keeping Up With the Kardashians|5139594|3600                  |3600            |1.0               |
|14719450  |7060037          |Mom                            |5445265|1740                

In [21]:
val user_program_metrics = spark.sql("SELECT USER,MASTER_TITLE,SUM(EPISODE_WATCH_DURATION)/SUM(EPISODE_DURATION) as PROGRAM_WATCH_RATIO, COUNT(DISTINCT PROGRAM_ID) as EPISODES_WATCHED FROM user_episode_watch_df GROUP BY USER,MASTER_TITLE")

user_program_metrics: org.apache.spark.sql.DataFrame = [USER: int, MASTER_TITLE: string ... 2 more fields]


In [23]:
user_program_metrics.createOrReplaceTempView("user_program_metrics")
//user_program_metrics.show(10,false)

2019-06-23 18:26:36 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:36 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:36 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:36 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:38 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:38 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:38 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2019-06-23 18:26:38 WARN  RowBasedKeyValueBatch:173 - Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [26]:
spark.sql("SELECT DISTINCT MASTER_TITLE FROM user_episode_df WHERE CATEGORY_ID IN (3,4,5,8)").count()

res15: Long = 1837


In [35]:
val user_program_prefernce = spark.sql("SELECT USER,PROGRAM_WATCH_RATIO,EPISODES_WATCHED,EPISODES_WATCHED/EPISODE_COUNT AS USER_WATCH_FREQUENCY ,PROGRAM_WATCH_RATIO *(EPISODES_WATCHED/EPISODE_COUNT) AS USER_PROGRAM_PREFERANCE_METRIC,t.* from user_program_metrics u join tivo_prog_df t ON t.MASTER_TITLE = u.MASTER_TITLE ORDER BY USER")

user_program_prefernce: org.apache.spark.sql.DataFrame = [USER: int, PROGRAM_WATCH_RATIO: double ... 23 more fields]


In [36]:
user_program_prefernce.printSchema()

root
 |-- USER: integer (nullable = true)
 |-- PROGRAM_WATCH_RATIO: double (nullable = true)
 |-- EPISODES_WATCHED: long (nullable = false)
 |-- USER_WATCH_FREQUENCY: double (nullable = true)
 |-- USER_PROGRAM_PREFERANCE_METRIC: double (nullable = true)
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable

In [56]:
user_program_prefernce.createOrReplaceTempView("user_program_prefernce")
user_program_prefernce.write.parquet("/Users/mthota/Dropbox/Data/final_datasets/feature_enginnering/user_program_features/stage/")

org.apache.spark.sql.AnalysisException:  path file:/Users/mthota/Dropbox/Data/final_datasets/feature_enginnering/user_program_features/stage already exists.;

In [58]:

val user_preference = spark.read.parquet("/Users/mthota/Dropbox/Data/final_datasets/feature_enginnering/user_program_features/stage/")
user_program_prefernce.createOrReplaceTempView("user_program_prefernce")

user_preference: org.apache.spark.sql.DataFrame = [USER: int, PROGRAM_WATCH_RATIO: double ... 23 more fields]


In [59]:
val user_program_prefernce_matrix = spark.sql("SELECT u.*,imdb_program_id,imdb_genres,imdb_titleType,imdb_averageRating,imdb_numVotes,imdb_cast_details,wiki_summaries from user_program_prefernce u left join tivo_imdb_wiki_df t on u.MASTER_TITLE=t.MASTER_TITLE")

user_program_prefernce_matrix: org.apache.spark.sql.DataFrame = [USER: int, PROGRAM_WATCH_RATIO: double ... 30 more fields]


In [60]:
user_program_prefernce_matrix.printSchema()

root
 |-- USER: integer (nullable = true)
 |-- PROGRAM_WATCH_RATIO: double (nullable = true)
 |-- EPISODES_WATCHED: long (nullable = false)
 |-- USER_WATCH_FREQUENCY: double (nullable = true)
 |-- USER_PROGRAM_PREFERANCE_METRIC: double (nullable = true)
 |-- MASTER_TITLE: string (nullable = true)
 |-- EVENT_DATE: string (nullable = true)
 |-- CATEGORY_ID: string (nullable = true)
 |-- SUBCATEGORY_ID: string (nullable = true)
 |-- RUNTIME: string (nullable = true)
 |-- EPISODE_COUNT: string (nullable = true)
 |-- day: string (nullable = true)
 |-- week_text: string (nullable = true)
 |-- week_encoded: string (nullable = true)
 |-- month: string (nullable = true)
 |-- vwp_days: string (nullable = true)
 |-- weekend: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- weekOfTheYear: string (nullable = true)
 |-- EVENT_END_DATE: string (nullable = true)
 |-- airStartDate: string (nullable = true)
 |-- airStartTime: string (nullable = true)
 |-- airEndDate: string (nullable

In [64]:
    user_preference.repartition(1).write.parquet("/Users/mthota/Dropbox/Data/final_datasets/feature_enginnering/user_program_features/stage2/")